In [1]:
import pandas as pd
import numpy as np
import sys
from scipy.sparse import lil_matrix
import scipy as scp
import time
%load_ext Cython

In [2]:
ITEM_NUM = 4318201

def get_logs_from_hardisk(path):
    f = open(path, 'r')
    a = f.read()
    dict_name = eval(a)
    f.close()
    return dict_name


f = open('usersActivity_map.txt', 'r')
m = f.read()
user_times_map = eval(m)
f.close()

In [3]:
%%cython

import datetime
import math

cpdef calculate_matrix(mat, list user_logs, dict user_times_map):
    cdef int index, i1, i2, count
    cdef list item_log
    cdef tuple u
    
    count = 0
    for u in user_logs:
        count += 1
        if count % 1000 == 0:
            print('The %d'%count + ' users are finished.')
            print(datetime.datetime.now().strftime('%H:%M:%S'))
            
        item_log = u[1]   

        for index, i1 in enumerate(item_log):
            for i2 in item_log[(index+1): ]:
                weight = 1/(math.log(1+user_times_map[u[0]]))
                mat[i1, i2] += weight
                mat[i2, i1] += weight
    return mat

In [4]:
user_logs = get_logs_from_hardisk('full_logs/userlogs_group3.txt')
f = open('upward_map.txt','r')
upward_map = eval(f.read())
f.close()
for u in user_logs:
    user_logs[u] = [int(upward_map[x]) for x in user_logs[u]]
user_logs = list(user_logs.items())


for i in range(0, len(user_logs), 10000):
    print('The %d '%i + ' batch is started!')
    print('--------------------------------')
    mat = lil_matrix((ITEM_NUM+1, ITEM_NUM+1), dtype=float)
    mat = calculate_matrix(mat, user_logs[i: i + 10000], user_times_map)
    scp.sparse.save_npz('tmpData/sparse_matrix_%d_batch_group3.npz'%i, mat.tocsr())
    print('save successfully')
    print('--------------------------------')

The 0  batch is started!
--------------------------------
The 1000 users are finished.
17:42:50
The 2000 users are finished.
17:43:20
The 3000 users are finished.
17:43:52
The 4000 users are finished.
17:44:22
The 5000 users are finished.
17:44:59
The 6000 users are finished.
17:45:46
The 7000 users are finished.
17:46:36
The 8000 users are finished.
17:47:37
The 9000 users are finished.
17:48:41
The 10000 users are finished.
17:49:35
save successfully
--------------------------------
The 10000  batch is started!
--------------------------------
The 1000 users are finished.
17:53:02
The 2000 users are finished.
17:53:50
The 3000 users are finished.
17:54:37
The 4000 users are finished.
17:55:29
The 5000 users are finished.
17:56:14
The 6000 users are finished.
17:57:02
The 7000 users are finished.
17:57:52
The 8000 users are finished.
17:58:46
The 9000 users are finished.
17:59:42
The 10000 users are finished.
18:00:31
save successfully
--------------------------------
The 20000  batch

The 9000 users are finished.
19:31:14
The 10000 users are finished.
19:31:18
save successfully
--------------------------------
The 170000  batch is started!
--------------------------------
The 1000 users are finished.
19:31:46
The 2000 users are finished.
19:31:50
The 3000 users are finished.
19:31:53
The 4000 users are finished.
19:31:56
save successfully
--------------------------------
